<a href="https://colab.research.google.com/github/marcelofschiavo/ds-cookbook/blob/main/04_Modelagem_Estatistica_(Inferencial).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Conceitos: Provar se seus insights são "reais".

"Receitas" (Códigos):

Correlação vs. Causalidade (Explicação em Markdown).

Teste-T: Como rodar (stats.ttest_ind) e como interpretar o p-value.

Teste Qui-Quadrado (Chi-Square): Como rodar (stats.chi2_contingency) e como interpretar (para ver se departamento tem relação com turnover).

ANOVA: (Bônus, se quiser) Para comparar 3+ grupos.